# Redes Neuronales Recurrentes

En este notebook, exploraremos uno de los tipos de Redes Neuronales Recurrentes (RNN) más utilizados: las Long Short-Term Memory (LSTM).
Estas están diseñadas para manejar **secuencias de datos** y resolver el problema del **desvanecimiento del gradiente** de las redes recurrentes básicas.

Como ya hemos visto en teoría, en una LSTM propagamos entre cada etapa dos vectores diferentes, el de estado y la salida en si. Para obtener dichos vectores, necesitamos calcular el valor de la puerta de actualización, la de olvido y la de salida.

Este proceso será transparente a nosotros dado que Keras ya nos proporciona una capa [`LSTM`](https://keras.io/api/layers/recurrent_layers/lstm/) y simplemente tendremos que preocuparnos de manejar la secuencia de entrada y la de salida.

## Conjunto de datos
Puesto que ahora trabajaremos con secuencias, para este ejemplo vamos a utilizar un conjunto de datos de texto. En este caso hemos optado por descargar el libro "Trafalgar" de Benito Pérez Galdós.

Como hemos explicado en teoría, existen muchos tipos de problemas que podemos resolver con secuencias, pero en este caso nos centraremos en una de las más sencillas de implementar: predecir la siguiente palabra en una secuencia de 5 palabras dadas.

### Descargar conjunto

In [ ]:
import requests

# Descargar el texto de "Trafalgar" de Benito Pérez Galdós
url = "https://www.gutenberg.org/cache/epub/16961/pg16961.txt"
response = requests.get(url, timeout=30)
text = response.text

### Preprocesar texto

Normalmente los conjuntos de datos contienen errores o elementos no deseados, en esta parte los eliminaremos. Algunos ejemplos típicos son los números, los saltos de línea `\n` o los tabuladores `\t`. <br>
Si abres la url del libro en tu navegador verás que al inicio y al final nos aparece un aviso de copyright que tendremos que eliminar.

En este punto verás que también separamos el texto en frases haciendo uso de la librería de texto `nltk`.

In [ ]:
! pip install unidecode
! pip install nltk

In [ ]:
import re
import time
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from unidecode import unidecode

# Eliminar saltos de línea y retornos de carro
text = re.sub(r'[\n\r]+', ' ', text)
# Eliminamos cabecera y fin en inglés
text = text.split("FIN DE TRAFALGAR")[0]
text = text.split(" -I- ")[1]
# Dividir el texto en frases utilizando la librería nltk
sentences = sent_tokenize(text)
# Eliminar acentos utilizando la librería unidecode
sentences = [unidecode(s) for s in sentences]
# Pasar a minúsculas y eliminar resto de caracteres extraños (dos puntos, punto y coma, números...)
sentences = [re.sub(r'[^a-z\s]', '', s.lower()) for s in sentences]

### Obtener vocabulario

Una vez tenemos todas las frases, para codificar cada una de las palabras del texto es necesario obtener el llamado `corpus` o `vocabulario`. Este nos sirve para saber que palabras únicas (sin repetición) aparecen en nuestro texto, lo cual es útil para asignar un índice a cada una. Como verás, en este caso asignamos el índice en función de la frecuencia de aparición, por tanto, la palabra más frecuente tendrá el índice más bajo.

Como recordarás, nuestro objetivo es que, dado un texto de máximo 5 palabras, el modelo entrenado sea capaz de continuar la frase hasta que decida finalizarla. Para lograrlo tenemos que tener en cuenta varios escenarios:
* ¿Y si nos dan menos de 5 palabras?
* ¿Como hacemos para detectar que la palabra generada por el modelo es la última de la frase?
* ¿Si nos dan una palabra que no existe en nuestro vocabulario? Una en otro idioma, por ejemplo.

Para solventar estos problemas, vamos a añadir tres palabras nuevas o `tokens` a nuestro vocabulario:
* **\<pad\>**: Token que representa *padding*, es decir, si nos dan menos de 5 palabras, rellenaremos las que falten con este padding.
* **\<eos\>**: Token que representa *end-of-sequence*, es decir, fin de secuencia. Si el modelo predice este token, sabremos que ya se acabó la frase y podemos detener la generación.
* **\<unk\>**: Token que representa *unknown*, es decir, desconocido. Si una palabra no aparece en el vocabulario, el modelo la codificará con este token.


In [ ]:
from collections import Counter

# Crear una lista con todas las palabras del texto
words = " ".join(sentences).split()
# Obtener el corpus del texto (todas las palabras que aparecen y su frecuencia)
word_counts = Counter(words)
# Filtrar palabras que aparecen menos de 2 veces, dado que no aportan mucho
word_counts = {word: count for word, count in word_counts.items() if count >=5}
# Ordenamos el corpus por frecuencia
word_counts = dict(sorted(word_counts.items(), key=lambda x: x[1], reverse=True))
# Extraemos solo las claves del diccionario anterior para crear el corpus
vocab = list(word_counts.keys())
# Añadimos tokens especiales <pad> (padding), <eos> (end of sequence), y <unk> (unknown)
vocab = ["<pad>", "<unk>", "<sos>", "<eos>"] + vocab
# Generamos un diccionario (y su inverso) donde asignamos un id a cada palabra según su frecuencia.
# La palabra más frecuente será la 1. El 0 lo dejamos para el padding
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for i, word in enumerate(vocab)}
vocab_size = len(vocab)

print(f'Tamaño del vocabulario: {vocab_size}')
print(f'Word to Index mapping: {word_to_ix}')

### Análisis de datos
En todos los problemas de aprendizaje, es muy importante hacer un análisis de los datos con los que vamos a trabajar, lo cual nos puede servir para diagnosticar o evitar futuros problemas.<br>
En este caso realizamos un simple análisis de la frecuencia de las palabras.

Como siempre suele suceder en estos casos, las palabras más comunes son las llamadas `stop-words`. Estas palabras son las que más solemos repetir en nuestros textos y las forman los artículos, las preposiciones, las conjunciones, ...

In [ ]:
import matplotlib.pyplot as plt

# Visualización de las 20 palabras más comunes
words_x = list(word_counts.keys())[:20]
counts_y = list(word_counts.values())[:20]

plt.figure(figsize=(10, 6))
plt.bar(words_x, counts_y)
plt.title('Palabras más comunes')
plt.xlabel('Palabras')
plt.ylabel('Frecuencia')
plt.show()

### Crear dataset
Una vez tenemos nuestro conjunto limpio y analizado, podemos crear los ejemplos con los que alimentaremos nuestro modelo. Como habíamos comentado queremos intentar completar una frase dadas, como máximo 5 palabras.<br>

En el siguiente código crearemos los ejemplos a partir de las frases del texto descargado. Estos ejemplos han de reflejar todos los escenarios a los que luego queremos que nuestro modelo pueda enfrentarse.

Por tanto, para cada frase, nuestro `Dataset` creará los siguientes ejemplos:

* Frase: *Frase de ejemplo*
    * [\<pad\>, \<pad\>, \<pad\>, \<pad\>, \<sos\>, frase], de
    * [\<pad\>, \<pad\>, \<pad\>, \<sos\>, frase, de], ejemplo
    * [\<pad\>, \<pad\>, \<sos\>, frase, de, ejemplo], \<eos\>

El primer vector es la secuencia de entrada y el elemento del final es la salida deseada. Ten en cuenta que al modelo no le damos el texto, le daremos los índices de cada palabra.

Además, para manejar los tokens de padding correctamente, se generará una máscara binaria correspondiente que indica cuáles son tokens válidos y cuáles son de padding.

Si nuestro `word_to_ix` fuese este:
```python
word_to_ix = {"<pad>": 0, "<sos>": 1, "<eos>": 2, "frase": 3, "de": 4, "ejemplo": 5}
```

Nuestros ejemplos quedarían así:
* [0, 0, 0, 0, 1, 3], 4
* [0, 0, 0, 1, 3, 4], 5
* [0, 0, 1, 3, 4, 5], 2

Y las máscaras correspondientes serían:

* [0, 0, 0, 0, 0, 1]
* [0, 0, 0, 0, 1, 1]
* [0, 0, 0, 1, 1, 1]

Estas máscaras aseguran que el modelo ignore los tokens de padding al procesar las secuencias.

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

# Función para convertir una oración en una secuencia de índices
def sentence_to_indices(sentence, word_to_ix, seq_length):
    unk_index = word_to_ix["<unk>"]
    words = ['<sos>'] + sentence.split() + ['<eos>']

    input_data = []
    output_data = []
    mask_data = []

    for i in range(len(words) - 1):
        seq_in = words[max(0, i + 1 - seq_length):i + 1]
        seq_in = ['<pad>'] * (seq_length - len(seq_in)) + seq_in
        seq_out = words[i + 1]

        seq_in_indices = [word_to_ix.get(word, unk_index) for word in seq_in]
        seq_out_index = word_to_ix.get(seq_out, unk_index)

        mask = [1 if word != '<pad>' else 0 for word in seq_in]

        if seq_out_index != unk_index:
            input_data.append(seq_in_indices)
            output_data.append(seq_out_index)
            mask_data.append(mask)

    return input_data, mask_data, output_data

# Generar dataset completo
def generate_dataset(sentences, word_to_ix, seq_length):
    # Para cada frase, generar los ejemplos
    x_data = []
    y_data = []
    mask_data = []

    for sentence in sentences:
        input_data, mask, output_data = sentence_to_indices(sentence, word_to_ix, seq_length)
        x_data.extend(input_data)
        y_data.extend(output_data)
        mask_data.extend(mask)

    return tf.data.Dataset.from_tensor_slices(((x_data, mask_data), y_data))

# Dividimos el conjunto en entrenamiento, validación y test (80%,10%,10%) de forma aleatoria.
sentences_train, sentences_test  = train_test_split(sentences, test_size=.2)
sentences_val, sentences_test = train_test_split(sentences_test, test_size=.5)

# Definir parámetros y datos
seq_length = 10
batch_size = 32

# Generar dataset
train_dataset = generate_dataset(sentences_train, word_to_ix, seq_length).batch(batch_size)
val_dataset = generate_dataset(sentences_val, word_to_ix, seq_length).batch(batch_size)
test_dataset = generate_dataset(sentences_test, word_to_ix, seq_length).batch(batch_size)

## Modelo
Para este problema crearemos el modelo más sencillo posible:
* **Capa Embedding**: Para cada elemento de la entrada, aprenderá un embedding que lo representará en un espacio de dimensión `embed_size`.
* **Capa LSTM**: Procesa el embedding de cada elemento de la secuencia de entrada, lo proyecta en otro espacio de tamaño `hidden_size` y retorna la secuencia de salida.
    * Ojo, la capa LSTM retorna, por defecto, la salida en el último instante de tiempo. Si quieres concatenar dos LSTM tendrás que indicar en la primera `return_sequences=True`.
    * Como ya hemos comentado en teoría, en estas redes se acumula toda la información de la secuencia en ese último elemento.
    * La máscara se pasa a la LSTM con el fin de que procese solo los elementos relevantes de la secuencia (no el padding).
* **Capa Linear**: Proyecta el vector del último elemento de la salida de la LSTM en un espacio de tamaño `output_size`. En este caso `output_size` es igual al tamaño del vocabulario dado que estamos intentando resolver un problema de multi-clasificación.


In [ ]:
from tensorflow.keras import layers, Model, Input

# Hiperparámetros del modelo
embed_size = 64
hidden_size = 32
output_size = vocab_size
num_epochs = 50
learning_rate = 0.0005

# Definir el modelo usando la API funcional
input_seq = Input(shape=(seq_length,), dtype=tf.int32, name='input_seq')
input_mask = Input(shape=(seq_length,), dtype=tf.int32, name='input_mask')
embedding_layer = layers.Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=seq_length)(input_seq)
lstm_layer = layers.LSTM(hidden_size, activation="relu")(embedding_layer, mask=input_mask)
output_layer = layers.Dense(output_size, activation="softmax")(lstm_layer)
model = Model(inputs=[input_seq, input_mask], outputs=output_layer)

# Compilar el modelo (necesario en TensorFlow/Keras)
model.compile(optimizer=tf.optimizers.Adam(learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Mostrar resumen del modelo
model.summary()

## Entrenamiento

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=num_epochs)

## Prueba

In [ ]:
import re
from unidecode import unidecode
import tensorflow as tf

# Definir la función de predicción
def predict(model, text, word_to_ix, ix_to_word, seq_length, max_len=20):
    # Preprocesar el texto de entrada
    text = unidecode(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = ["<sos>"] + text.split()

    # Convertir palabras a índices
    input_sequence = [word_to_ix.get(word, word_to_ix['<pad>']) for word in words]
    input_sequence = input_sequence[-seq_length:]  # Tomar solo las últimas seq_length palabras
    input_sequence = [word_to_ix['<pad>']] * (seq_length - len(input_sequence)) + input_sequence

    # Crear la máscara para la secuencia de entrada
    input_mask = [1 if word != word_to_ix['<pad>'] else 0 for word in input_sequence]

    # Convertir la secuencia y la máscara a tensores
    input_tensor = tf.constant([input_sequence], dtype=tf.int64)
    mask_tensor = tf.constant([input_mask], dtype=tf.int64)

    # Inicializar la lista de palabras generadas
    generated_words = words[1:]
    predicted_word = ""

    for _ in range(max_len):
        # Hacer una predicción usando el modelo
        output = model([input_tensor, mask_tensor], training=False)

        # Obtener la palabra con la probabilidad más alta
        predicted_index = tf.argmax(output, axis=-1).numpy()[0]
        predicted_word = ix_to_word[predicted_index]

        if predicted_word == '<eos>':
            break

        # Añadir la palabra generada a la lista
        generated_words.append(predicted_word)

        # Actualizar la secuencia de entrada y la máscara para la siguiente predicción
        input_sequence = input_sequence[1:] + [predicted_index]
        input_mask = input_mask[1:] + [1]

        input_tensor = tf.constant([input_sequence], dtype=tf.int64)
        mask_tensor = tf.constant([input_mask], dtype=tf.int64)

    return ' '.join(generated_words)

# Ejemplo de texto de entrada
input_text = "la batalla de"
predicted_text = predict(model, input_text, word_to_ix, ix_to_word, seq_length)
print(predicted_text)